# Programming Assignment 5 - Build your own Conventional Neural Network

After completing this project, you will be able to do the following:

- Build neural network conveniently.
- Configure with different regularization methods: Dropout, l1 or l2 regulation.

- **574 Only**: Fine tune pre-trained model to build your own projects.

# Dataset

This dataset is provided by our current research project, which utilize 20 kHz acoustic sensing to sense ASL gestures. All the 10 ASL words perform by 5 subjects. All images are generated by using the short-time Fourier transform (STFT) to calculate a spectrogram as the feature representation of the reflected near-ultrasound waves. Based on the Doppler effect, sign language gestures, including both hands and arms, will cause phase and frequency changes of the reflected sonic wave. The spectrogram contains information in both frequency and time domains. The spectrogram is also defined as the Power Spectral Density of the function:
\begin{equation}
    \textrm{spectrogram}\{x(t)\}(\tau,\omega)\equiv |X(\tau,\omega)|^{2}= \left| \sum^{\infty}_{n=-\infty}x[n]\omega[n-m]e^{-j\omega n}\right|^{2}
\end{equation}
where $x[n]$ is input signal, and $\omega[n-m]$ represents the overlapping Kaiser window function with an adjustable shape factor $\beta$ that improves the resolution and reduces the spectral leakage close to the sidelobes of the signal. The coefficients of the Kaiser window are computed as:
\begin{equation}
    \omega[n]=\frac{I_{0}\left(\beta\sqrt{1-\left(\frac{n-N/2}{N/2}\right)^{2}}\right)}{I_{0}(\beta)}, 0 \leq n \leq N
\end{equation}

This dataset has a training set of 5,000 examples, and a test set of 1,000 examples.

# Submission

1. **Coding checks (60 points)** - The code for your implementation should be in Python only. The name of the Main file should be main.ipynb or main.py. Please provide necessary comments in the code.

2. **Written Report (40 points)**: The report should be delivered as a separate pdf file, and it is recommended for you to use the NIPS template to structure your report. You may include comments in the Jupyter Notebook, however you will need to duplicate the results in the report. The report should describe your results, experimental setup and comparison between the results obtained
from different setting of the algorithm and dataset.**Again, the questions in the Assignment PDF and here are the same (for the written report), we just put them in both places for convenience.**


As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 3 things:
- Your completed jupyter notebook.
- Your written report.

In [89]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Activation, Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
# YOUR CODE HERE


## Part 1. loading dataset

We have splitted the dataset into training, validation and test. You can load every single folder to load dataset using keras ImageDataGenerator (10 points).

In [90]:
# load the file from our dataset including training, validation and testing part
img_width, img_height = 224, 224
train_data_dir      = r'C:\Users\mahes\Desktop\UB\Spring-22\Intro to ML\PA-5\pa5\pa5\pictures\pictures\train'
validation_data_dir = r'C:\Users\mahes\Desktop\UB\Spring-22\Intro to ML\PA-5\pa5\pa5\pictures\pictures\val'
test_data_dir       = r'C:\Users\mahes\Desktop\UB\Spring-22\Intro to ML\PA-5\pa5\pa5\pictures\pictures\test'

epochs = 50
batch_size = 16

# # this is a generator that will read pictures found in
# # subfolers of 'data/train', and indefinitely generate
# # batches of augmented image data
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Your Code HERE
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size)
valid_generator = train_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size)
test_generator  = test_datagen.flow_from_directory(test_data_dir, target_size=(img_width, img_height), batch_size=batch_size)


# please print the number of samples in each folder 
print(train_generator.samples)
print(valid_generator.samples)
print(test_generator.samples)

Found 4176 images belonging to 10 classes.
Found 1392 images belonging to 10 classes.
Found 1392 images belonging to 10 classes.
4176
1392
1392


In [91]:
for itr in train_generator:
    print(itr[1].shape)

(16, 10)
(16, 10)
(16, 10)
(16, 10)
(16, 10)
(16, 10)
(16, 10)
(16, 10)
(16, 10)


KeyboardInterrupt: 

## Part 2.1 - Build your Neural Network and Train

Build a Convolutional Neural Network with 2 or 3 hidden layers without regularization methods, which includes Conv2D layer, activation Layer. please use training dataset and validation dataset for training processs, and plot the training process with Loss trend and accuracy trend (30 Points).

## Part 2.2 - Test 

Test your machine learning model on the testing set: After finishing all the above steps, fix your hyper-parameters(learning rate, number of neurons per layer) and model parameter and test your model’s performance on the testing set. This shows the effectiveness of your model’s generalization power gained by learning. For test dataset, the performance should be more than 80% (10 Points).

In [92]:
from tensorflow.keras.applications import InceptionV3

In [95]:
# build conv2D CNN model, be careful with softmax and output layers is 10

# define the input shape of Convolutional Neural Network
# Your Code HERE
input_shape = (img_width, img_height) 
num_classes = 10


# MODEL - 1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

'''
# MODEL-2
# define the Convolutional Neural Network
model = keras.Sequential(
    [
        Conv2D(32, kernel_size=(3, 3), input_shape = input_shape, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ]
)
'''
'''
# MODEL - 3
base_model = InceptionV3(input_shape=(img_height, img_width, 3), weights='imagenet', include_top=False)
model=Sequential()

model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
'''
model.summary()


Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_787 (Conv2D)          (None, 222, 222, 32)      896       
_________________________________________________________________
activation_778 (Activation)  (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_788 (Conv2D)          (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_779 (Activation)  (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_64 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_789 (Conv2D)          (None, 52, 52, 64)      

In [ ]:
batch_size = 128
epochs = 20

# Compiling the model and training including the files of compile and fit
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#Train the model with training and validation set
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
model.fit(train_generator, steps_per_epoch=2000 // batch_size, epochs=epochs, validation_data=valid_generator, validation_steps=800 // batch_size)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 15 steps, validate for 6 steps
Epoch 1/20
15/15 [==============================] - 10s 636ms/step - loss: 2.3414 - accuracy: 0.1042 - val_loss: 2.2859 - val_accuracy: 0.1875
Epoch 2/20
15/15 [==============================] - 11s 705ms/step - loss: 2.2678 - accuracy: 0.1125 - val_loss: 2.2261 - val_accuracy: 0.1875
Epoch 3/20
15/15 [==============================] - 10s 644ms/step - loss: 2.1674 - accuracy: 0.1583 - val_loss: 2.1279 - val_accuracy: 0.2188
Epoch 4/20
15/15 [==============================] - 10s 668ms/step - loss: 2.1077 - accuracy: 0.1667 - val_loss: 2.0321 - val_accuracy: 0.3229
Epoch 5/20
15/15 [==============================] - 10s 681ms/step - loss: 2.0272 - accuracy: 0.2500 - val_loss: 1.9468 - val_accuracy: 0.4375
Epoch 6/20
15/15 [==============================] - 9s 621ms/step - loss: 1.9545 - accuracy: 0.2708 - val_loss: 1.9152 - val_accuracy: 0.3854
Epoch 7/20
15/15 [==============================] - 

%Test

In [ ]:
# Test the model on a testing dataset
loss_value , accuracy = model.evaluate(test_generator)
print(loss_value,accuracy)

## Part 2.3 - L1 Regularization. Please add L1 regularization setting in your Conv2D layer. Then, train your new model separately, and plot the training process including loss and accuracy. (10 points)

In [ ]:
# please redefine your model with setting the L1 Regularization in the layer of Conv 2D

# Your code HERE


## Part 2.4 - L2 Regularization. Please add L2 regularization setting in your Conv2D layer. Then, train your new model separately, and plot the training process including loss and accuracy. (10 points)

In [ ]:
#please redefine your model and set the L2 Regularization in the layer of Conv 2D

# Your code HERE



## Part3 - ** only for 574 students **.
1. Fine tune the well pre-trained model, Resnet 50, with different freeze layers. 
First,  load pre-trained resnet 50 from library.
Second, Fine-tune the model to fit our project, 10-classes.
Third,  freeze different layers, plot different training process with different frozen layers (at least three different layers).

In [ ]:
# load pre-trained resnet 50 from libarary
# your Code HERE


# Fine-tune the model to fit our project  
# your Code HERE


# define the freeze layers 
# your Code HERE


# Please compile the new model
#model.compile()


#Train the model with training and validation set
#model.fit()